# Add number of species to precalculated tables

## Import packages

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

## Gadm0

In [12]:
gadm0 = gpd.read_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/updates/gadm0_precalculated_20220224_backup20220818.geojson')

In [13]:
gadm0.head(1)

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,...,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,country_size,ObjectId,alternative_names,geometry
0,ABW,Aruba,1,181.9384,"[{""SliceNumber"":1819,""per_global"":0.6,""per_aoi...","[{""SliceNumber"":142,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":329,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":353,""per_global"":0.01,""per_aoi...",20.6093,115656.1,...,Shrubland,Tropical Dry,None,24.66,None,6.39,5,1,None,"POLYGON ((-69.97820 12.46986, -70.02847 12.503..."


In [14]:
gadm0.columns

Index(['GID_0', 'NAME_0', 'MOL_ID', 'AREA_KM2', 'amphibians', 'birds',
       'mammals', 'reptiles', 'percentage_protected', 'population_sum',
       'majority_land_cover_climate_reg', 'land_cover_majority',
       'climate_regime_majority', 'percent_irrigated', 'percent_rainfed',
       'percent_rangeland', 'percent_urban', 'country_size', 'ObjectId',
       'alternative_names', 'geometry'],
      dtype='object')

In [15]:
# Get data for all taxa
amphibians = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm0_amphibians.csv')
birds = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm0_birds.csv')
mammals = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm0_mammals.csv')
reptiles = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/Inputs/gadm0_reptiles.csv')

In [16]:
amphibians.head()

,OBJECTID,MOL_ID,SliceNumber,FREQUENCY,SUM
0,1,1,1819,1,151.0
1,2,1,4583,1,95.0
2,3,1,5543,1,152.0
3,4,2,32,4,317.0
4,5,2,950,43,44105.0


In [17]:
amph = amphibians.groupby('MOL_ID')['SliceNumber'].count().astype(int)
bir = birds.groupby('MOL_ID')['SliceNumber'].count().astype(int)
mamm = mammals.groupby('MOL_ID')['SliceNumber'].count().astype(int)
rept = reptiles.groupby('MOL_ID')['SliceNumber'].count().astype(int)

In [18]:
frame = { 'Amphibians': amph, 'Birds': bir, 'Mammals': mamm, 'Reptiles': rept }
df = pd.DataFrame(frame).reset_index()
cols = ['Amphibians', 'Birds', 'Mammals', 'Reptiles']
df[cols] = df[cols].fillna(0)
df[cols] = df[cols].astype('int')
df['nspecies'] = df['Amphibians'] + df['Birds'] + df['Mammals'] + df['Reptiles']
df

,MOL_ID,Amphibians,Birds,Mammals,Reptiles,nspecies
0,1,3,82,48,30,163
1,2,8,316,140,134,598
2,3,128,891,322,335,1676
3,4,2,96,6,10,114
4,5,5,88,26,2,121
...,...,...,...,...,...,...
250,251,0,36,0,3,39
251,252,6,218,71,129,424
252,253,117,706,271,399,1493
253,254,93,695,275,218,1281


In [19]:
df[df.nspecies==0]

,MOL_ID,Amphibians,Birds,Mammals,Reptiles,nspecies


In [20]:
gadm0_updated = gadm0.merge(df[['nspecies', 'MOL_ID']], how='left', on = 'MOL_ID')
gadm0_updated

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,...,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,country_size,ObjectId,alternative_names,geometry,nspecies
0,ABW,Aruba,1,1.819384e+02,"[{""SliceNumber"":1819,""per_global"":0.6,""per_aoi...","[{""SliceNumber"":142,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":329,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":353,""per_global"":0.01,""per_aoi...",20.609300,115656.10,...,Tropical Dry,None,24.66,None,6.39,5,1,None,"POLYGON ((-69.97820 12.46986, -70.02847 12.503...",163
1,AFG,Afghanistan,2,6.438575e+05,"[{""SliceNumber"":32,""per_global"":100.0,""per_aoi...","[{""SliceNumber"":26,""per_global"":1.44,""per_aoi""...","[{""SliceNumber"":167,""per_global"":11.15,""per_ao...","[{""SliceNumber"":1,""per_global"":28.71,""per_aoi""...",3.785337,30389700.00,...,Cool Temperate Dry,6.48,18.49,62.15,0.07,3,2,None,"POLYGON ((68.52644 31.75435, 68.53852 31.75457...",598
2,AGO,Angola,3,1.247422e+06,"[{""SliceNumber"":2,""per_global"":0.59,""per_aoi"":...","[{""SliceNumber"":26,""per_global"":4.67,""per_aoi""...","[{""SliceNumber"":28,""per_global"":2.22,""per_aoi""...","[{""SliceNumber"":40,""per_global"":26.68,""per_aoi...",6.679349,36074940.00,...,Sub Tropical Moist,0.07,4.19,67.89,0.08,2,3,None,"MULTIPOLYGON (((11.73347 -16.67255, 11.74014 -...",1676
3,AIA,Anguilla,4,8.330331e+01,"[{""SliceNumber"":1819,""per_global"":0.23,""per_ao...","[{""SliceNumber"":142,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":329,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":316,""per_global"":25.32,""per_ao...",2.880000,13601.96,...,Tropical Moist,None,NaN,None,NaN,5,4,None,"MULTIPOLYGON (((-63.42375 18.58903, -63.42847 ...",114
4,ALA,Åland,5,1.506261e+03,"[{""SliceNumber"":963,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":116,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":129,""per_global"":0.01,""per_aoi...","[{""SliceNumber"":2500,""per_global"":0.01,""per_ao...",1.449518,29158.96,...,Cool Temperate Moist,None,50.27,None,NaN,5,5,None,"MULTIPOLYGON (((21.32195 59.74986, 21.32472 59...",121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,XSP,Spratly Islands,251,1.874750e+00,None,"[{""SliceNumber"":141,""per_global"":0.0,""per_aoi""...",None,"[{""SliceNumber"":3883,""per_global"":0.0,""per_aoi...",0.000000,0.00,...,None,None,NaN,None,NaN,5,251,None,"MULTIPOLYGON (((114.35472 11.45139, 114.35167 ...",39
251,YEM,Yemen,252,4.522034e+05,"[{""SliceNumber"":974,""per_global"":74.01,""per_ao...","[{""SliceNumber"":26,""per_global"":0.26,""per_aoi""...","[{""SliceNumber"":33,""per_global"":14.12,""per_aoi...","[{""SliceNumber"":9,""per_global"":2.12,""per_aoi"":...",0.000000,30204360.00,...,Tropical Desert,0.98,4.81,59.46,0.52,3,252,None,"MULTIPOLYGON (((53.26931 12.12875, 53.31011 12...",424
252,ZAF,South Africa,253,1.219710e+06,"[{""SliceNumber"":33,""per_global"":43.92,""per_aoi...","[{""SliceNumber"":26,""per_global"":3.15,""per_aoi""...","[{""SliceNumber"":28,""per_global"":3.91,""per_aoi""...","[{""SliceNumber"":40,""per_global"":0.95,""per_aoi""...",8.157492,59911770.00,...,Warm Temperate Dry,1.22,27.51,68.14,0.69,2,253,None,"MULTIPOLYGON (((19.42097 -34.67903, 19.42208 -...",1493
253,ZMB,Zambia,254,7.504919e+05,"[{""SliceNumber"":34,""per_global"":36.77,""per_aoi...","[{""SliceNumber"":26,""per_global"":3.52,""per_aoi""...","[{""SliceNumber"":28,""per_global"":10.72,""per_aoi...","[{""SliceNumber"":40,""per_global"":11.6,""per_aoi""...",39.120460,18692100.00,...,Sub Tropical Dry,0.11,13.44,55.89,0.20,3,254,None,"POLYGON ((25.88753 -17.98407, 25.88284 -17.984...",1281


In [25]:
gadm0_updated[gadm0_updated.nspecies==0]

,GID_0,NAME_0,MOL_ID,AREA_KM2,amphibians,birds,mammals,reptiles,percentage_protected,population_sum,...,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,country_size,ObjectId,alternative_names,geometry,nspecies


In [28]:
gadm0_updated.to_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm0/updates/gadm0_precalculated_20220224_nspecies.geojson', driver='GeoJSON')

## gadm1

In [50]:
gadm1 = gpd.read_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm1/update_nspecies/gadm1_precalculated_range_area_0_backup20220818.geojson')

In [30]:
gadm1.head(1)

,GID_0,NAME_0,GID_1,NAME_1,MOL_ID,AREA_KM2,reptiles,amphibians,mammals,birds,...,percent_rainfed,percent_rangeland,percent_urban,population_sum,majority_land_cover_climate_reg,land_cover_majority,climate_regime_majority,country_size,ObjectId,geometry
0,LBN,Lebanon,LBN.1_1,Akkar,1597,791.1171,"[{""SliceNumber"":2,""per_global"":0.75,""per_aoi"":...","[{""SliceNumber"":955,""per_global"":0.03,""per_aoi...","[{""SliceNumber"":259,""per_global"":0.02,""per_aoi...","[{""SliceNumber"":141,""per_global"":0.0,""per_aoi""...",...,71.34,NaN,None,195103.0,173.0,Cropland,Warm Temperate Moist,5,1,"MULTIPOLYGON (((35.74834 34.23963, 35.74798 34..."


In [32]:
gadm1.columns

Index(['GID_0', 'NAME_0', 'GID_1', 'NAME_1', 'MOL_ID', 'AREA_KM2', 'reptiles',
       'amphibians', 'mammals', 'birds', 'percentage_protected',
       'percent_irrigated', 'percent_rainfed', 'percent_rangeland',
       'percent_urban', 'population_sum', 'majority_land_cover_climate_reg',
       'land_cover_majority', 'climate_regime_majority', 'country_size',
       'ObjectId', 'geometry'],
      dtype='object')

In [51]:
# Get data for all taxa
amphibians = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm1/Inputs/amphibians_gadm1_final_20211003_0.csv')
birds = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm1/Inputs/birds_gadm1_final_0.csv')
mammals = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm1/Inputs/mammals_gadm1_final_0.csv')
reptiles = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/gadm1/Inputs/reptiles_gadm1_final_20211003_0.csv')

In [53]:
amphibians.head()

,OID,MOL_ID,SliceNumbe,FREQUENCY,SUM_amphib
0,1,1,951,3,158
1,2,2,1707,3,5508
2,3,6,1707,1,235
3,4,7,950,11,13691
4,5,7,1707,12,29675


In [41]:
reptiles.columns

Index(['OID', 'MOL_ID', 'SliceNumbe', 'FREQUENCY', 'SUM_reptil'], dtype='object')

In [54]:
amph = amphibians.groupby('MOL_ID')['SliceNumbe'].count().astype(int)
bir = birds.groupby('MOL_ID')['SliceNumber'].count().astype(int)
mamm = mammals.groupby('MOL_ID')['SliceNumber'].count().astype(int)
rept = reptiles.groupby('MOL_ID')['SliceNumbe'].count().astype(int)

In [55]:
frame = { 'Amphibians': amph, 'Birds': bir, 'Mammals': mamm, 'Reptiles': rept }
df = pd.DataFrame(frame).reset_index()
cols = ['Amphibians', 'Birds', 'Mammals', 'Reptiles']
df[cols] = df[cols].fillna(0)
df[cols] = df[cols].astype('int')
df['nspecies'] = df['Amphibians'] + df['Birds'] + df['Mammals'] + df['Reptiles']
df

,MOL_ID,Amphibians,Birds,Mammals,Reptiles,nspecies
0,1,1,192,77,46,316
1,2,1,134,51,42,228
2,3,0,163,53,37,253
3,4,0,140,50,48,238
4,5,0,125,38,19,182
...,...,...,...,...,...,...
3605,3606,39,488,158,108,793
3606,3607,45,524,171,147,887
3607,3608,44,507,164,123,838
3608,3609,40,488,167,136,831


In [56]:
df[df.nspecies==0]

,MOL_ID,Amphibians,Birds,Mammals,Reptiles,nspecies


In [57]:
gadm1_updated = gadm1.merge(df[['nspecies', 'MOL_ID']], how='left', on = 'MOL_ID')
gadm1_updated

,GID_0,NAME_0,GID_1,NAME_1,MOL_ID,AREA_KM2,reptiles,amphibians,mammals,birds,...,percent_rangeland,percent_urban,population_sum,majority_land_cover_climate_reg,land_cover_majority,climate_regime_majority,country_size,ObjectId,geometry,nspecies
0,LBN,Lebanon,LBN.1_1,Akkar,1597,791.1171,"[{""SliceNumber"":2,""per_global"":0.75,""per_aoi"":...","[{""SliceNumber"":955,""per_global"":0.03,""per_aoi...","[{""SliceNumber"":259,""per_global"":0.02,""per_aoi...","[{""SliceNumber"":141,""per_global"":0.0,""per_aoi""...",...,NaN,None,195103.0,173.0,Cropland,Warm Temperate Moist,5,1,"MULTIPOLYGON (((35.74834 34.23963, 35.74798 34...",218
1,IDN,Indonesia,IDN.19_1,Maluku,1194,46347.6000,"[{""SliceNumber"":94,""per_global"":2.07,""per_aoi""...","[{""SliceNumber"":1041,""per_global"":74.68,""per_a...","[{""SliceNumber"":27,""per_global"":0.01,""per_aoi""...","[{""SliceNumber"":34,""per_global"":0.14,""per_aoi""...",...,0.99,1.79,1974849.0,392.0,Forest,Tropical Moist,2,2,"MULTIPOLYGON (((130.92404 -8.25113, 130.91696 ...",618
2,LBN,Lebanon,LBN.2_1,Baalbak - Hermel,1598,2846.7640,"[{""SliceNumber"":2,""per_global"":2.1,""per_aoi"":7...","[{""SliceNumber"":955,""per_global"":0.06,""per_aoi...","[{""SliceNumber"":259,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":97,""per_global"":0.0,""per_aoi"":...",...,7.42,4.48,548145.1,184.0,Cropland,Warm Temperate Dry,5,3,"POLYGON ((36.01317 33.86936, 36.01034 33.86726...",239
3,IDN,Indonesia,IDN.18_1,Maluku Utara,1195,31508.5200,"[{""SliceNumber"":94,""per_global"":0.48,""per_aoi""...","[{""SliceNumber"":1037,""per_global"":100.0,""per_a...","[{""SliceNumber"":23,""per_global"":1.09,""per_aoi""...","[{""SliceNumber"":37,""per_global"":48.11,""per_aoi...",...,NaN,5.25,1539865.0,392.0,Forest,Tropical Moist,2,4,"MULTIPOLYGON (((128.68254 2.38017, 128.68059 2...",370
4,ECU,Ecuador,ECU.6_1,Cotopaxi,801,6172.3850,"[{""SliceNumber"":310,""per_global"":2.01,""per_aoi...","[{""SliceNumber"":555,""per_global"":24.8,""per_aoi...","[{""SliceNumber"":59,""per_global"":0.84,""per_aoi""...","[{""SliceNumber"":27,""per_global"":0.02,""per_aoi""...",...,62.57,None,487626.1,176.0,Forest,Warm Temperate Moist,4,5,"MULTIPOLYGON (((-78.40904 -0.72033, -78.40891 ...",961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3605,ZAF,South Africa,ZAF.9_1,Western Cape,3590,129473.0000,"[{""SliceNumber"":125,""per_global"":15.85,""per_ao...","[{""SliceNumber"":44,""per_global"":97.27,""per_aoi...","[{""SliceNumber"":48,""per_global"":99.55,""per_aoi...","[{""SliceNumber"":53,""per_global"":1.02,""per_aoi""...",...,81.66,0.84,7340086.0,183.0,Shrubland,Warm Temperate Dry,2,3606,"MULTIPOLYGON (((19.42097 -34.67903, 19.42208 -...",711
3606,ZMB,Zambia,ZMB.1_1,Central,3591,94553.7000,"[{""SliceNumber"":40,""per_global"":0.26,""per_aoi""...","[{""SliceNumber"":34,""per_global"":1.9,""per_aoi"":...","[{""SliceNumber"":28,""per_global"":1.49,""per_aoi""...","[{""SliceNumber"":26,""per_global"":0.45,""per_aoi""...",...,52.97,0,2224357.0,290.0,Forest,Sub Tropical Dry,3,3607,"POLYGON ((27.76537 -15.63295, 27.72829 -15.650...",889
3607,ZMB,Zambia,ZMB.2_1,Copperbelt,3592,31401.7200,"[{""SliceNumber"":40,""per_global"":0.69,""per_aoi""...","[{""SliceNumber"":65,""per_global"":3.32,""per_aoi""...","[{""SliceNumber"":28,""per_global"":0.51,""per_aoi""...","[{""SliceNumber"":26,""per_global"":0.15,""per_aoi""...",...,59.91,1.49,2585174.0,268.0,Forest,Sub Tropical Moist,3,3608,"POLYGON ((26.86187 -13.71910, 26.81332 -13.694...",758
3608,ZMB,Zambia,ZMB.3_1,Eastern,3593,50544.8300,"[{""SliceNumber"":40,""per_global"":0.25,""per_aoi""...","[{""SliceNumber"":34,""per_global"":11.35,""per_aoi...","[{""SliceNumber"":28,""per_global"":0.68,""per_aoi""...","[{""SliceNumber"":26,""per_global"":0.24,""per_aoi""...",...,45.37,0.36,2157454.0,274.0,Forest,Sub Tropical Dry,3,3609,"POLYGON ((30.20338 -14.96573, 30.19933 -14.826...",844


In [58]:
gadm1_updated.nspecies[gadm1_updated.NAME_1=='Galicia']

80    303
Name: nspecies, dtype: int64

In [59]:
gadm1_updated.to_file('/Users/sofia/Documents/HE_Data/Precalculated/gadm1/update_nspecies/gadm1_precalculated_20220224_nspecies.geojson', driver='GeoJSON')

## wdpa

In [2]:
wdpa = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/outputs/wdpa_precal_with_contextual_updated.csv')

In [3]:
wdpa.head(1)

,field_1,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,...,percent_rainfed,percent_rangeland,percent_urban,population_sum,WDPA_PID,ORIG_NA,STATUS,STATUS_,GOV_TYP,geometry
0,0,Boulder Beach,1,1.136031298847187,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,...,0.0,100.0,0.0,2.110001444816589,310492,Boulder Beach / WWF Block,Designated,,Federal or national ministry or agency,None


In [3]:
wdpa.columns

Index(['Unnamed: 0', 'NAME', 'MOL_ID', 'AREA_KM2', 'WDPAID', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'amphibians', 'birds',
       'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'land_cover_majority', 'climate_regime_majority', 'percent_irrigated',
       'percent_rainfed', 'percent_rangeland', 'percent_urban',
       'population_sum', 'WDPA_PID', 'ORIG_NA', 'STATUS', 'STATUS_',
       'GOV_TYP'],
      dtype='object')

In [3]:
# Get data for all taxa
amphibians = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/wdpa_amphibians_final_20211003.csv')
birds = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/wdpa_birds_final_20211003.csv')
mammals = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/wdpa_mammals_final_20211003.csv')
reptiles = pd.read_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/Inputs/wdpa_reptiles_final_20211003.csv')

In [7]:
reptiles.head()

,OBJECTID,MOL_ID,X,Y,reptiles,SliceNumber,Dimensions
0,1,121,169.34608,-44.652219,1.0,6163,SliceNumber
1,2,121,169.34608,-44.652219,1.0,6314,SliceNumber
2,3,121,169.34608,-44.652219,1.0,6478,SliceNumber
3,4,121,169.34608,-44.652219,1.0,6481,SliceNumber
4,5,121,169.34608,-44.652219,1.0,6488,SliceNumber


In [4]:
amph = amphibians.groupby('MOL_ID')['SliceNumber'].count().astype(int)
bir = birds.groupby('MOL_ID')['SliceNumber'].count().astype(int)
mamm = mammals.groupby('MOL_ID')['SliceNumber'].count().astype(int)
rept = reptiles.groupby('MOL_ID')['SliceNumber'].count().astype(int)

In [5]:
frame = { 'Amphibians': amph, 'Birds': bir, 'Mammals': mamm, 'Reptiles': rept }
df = pd.DataFrame(frame).reset_index()
cols = ['Amphibians', 'Birds', 'Mammals', 'Reptiles']
df[cols] = df[cols].fillna(0)
df[cols] = df[cols].astype('int')
df['nspecies'] = df['Amphibians'] + df['Birds'] + df['Mammals'] + df['Reptiles']
df

,MOL_ID,Amphibians,Birds,Mammals,Reptiles,nspecies
0,1,0,49,3,0,52
1,2,2,68,2,6,78
2,3,2,24,2,2,30
3,4,2,62,2,4,70
4,5,2,48,0,3,53
...,...,...,...,...,...,...
205893,217453,2,95,25,2,124
205894,217455,2,76,25,2,105
205895,217456,0,40,2,0,42
205896,217457,6,59,40,3,108


In [6]:
wdpa['MOL_ID'] = wdpa['MOL_ID'].astype(int)

In [11]:
wdpa_updated = wdpa.merge(df[['nspecies', 'MOL_ID']], how='left', on = 'MOL_ID')
wdpa_updated

,Unnamed: 0,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,...,percent_rainfed,percent_rangeland,percent_urban,population_sum,WDPA_PID,ORIG_NA,STATUS,STATUS_,GOV_TYP,nspecies
0,0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,...,0.0,100.00,0.0,2.110001,310492,Boulder Beach / WWF Block,Designated,NaN,Federal or national ministry or agency,52.0
1,1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,0.0,0.00,0.0,1.315837,307797,Ferndale,Designated,NaN,Federal or national ministry or agency,78.0
2,2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,0.0,0.00,0.0,0.000000,307745,Broughton Bay,Designated,NaN,Federal or national ministry or agency,30.0
3,3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,0.0,100.00,0.0,0.000000,307867,Kaipupu Point,Designated,NaN,Federal or national ministry or agency,70.0
4,4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,...,0.0,83.33,0.0,3.103363,303963,Catlins Conservation Park,Designated,NaN,Federal or national ministry or agency,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217481,217481,Sablayan Fish Sanctuary,217482,0.000000,555716517.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,0.0,0.00,0.0,0.000000,555716517,Sablayan Fish Sanctuary,Established,2000.0,Not Reported,NaN
217482,217482,Gurion Marine Sanctuary,217483,0.000000,555716527.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,0.0,0.00,0.0,0.000000,555716527,Gurion Marine Sanctuary,Established,2002.0,Not Reported,NaN
217483,217483,Bontod-Maravilla Fish Sanctuary and Marine Res...,217484,0.000000,555716306.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,0.0,0.00,0.0,0.000000,555716306,Bontod-Maravilla Fish Sanctuary and Marine Res...,Established,2005.0,Not Reported,NaN
217484,217484,Bani Marine Protected Area,217485,0.000000,555716297.0,Locally-managed Marine Protected Area,Locally-managed Marine Protected Area,Not Applicable,OECM,PHL,...,0.0,0.00,0.0,0.000000,555716297,Bani Marine Protected Area,Established,2006.0,Not Reported,NaN


In [12]:
wdpa_updated['nspecies'] = wdpa_updated['nspecies'].fillna(0)
wdpa_updated['nspecies'] = wdpa_updated['nspecies'].astype('int')

In [13]:
wdpa_updated.nspecies[wdpa_updated.NAME=='Serra do Xistral']

132513    175
Name: nspecies, dtype: int64

In [10]:
wdpa_updated.to_csv('/Users/sofia/Documents/HE_Data/Precalculated/WDPA_Precalculated/outputs/wdpa_precal_with_contextual_nspecies.csv')

In [14]:
simp50 = gpd.read_file('/Users/sofia/Documents/HE_Data/WDPA/Simplifications/WDPA_Ter_Simp50/WDPA_Ter_Simp50.shp')

In [15]:
wdpa_updated.columns

Index(['Unnamed: 0', 'NAME', 'MOL_ID', 'AREA_KM2', 'WDPAID', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'amphibians', 'birds',
       'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'land_cover_majority', 'climate_regime_majority', 'percent_irrigated',
       'percent_rainfed', 'percent_rangeland', 'percent_urban',
       'population_sum', 'WDPA_PID', 'ORIG_NA', 'STATUS', 'STATUS_', 'GOV_TYP',
       'nspecies'],
      dtype='object')

In [16]:
simp50.columns

Index(['MOL_ID', 'WDPAID', 'WDPA_PID', 'NAME', 'DESIG', 'DESIG_T', 'IUCN_CA',
       'STATUS', 'GOV_TYP', 'MANG_AU', 'ISO3', 'AREA_KM2', 'geometry'],
      dtype='object')

In [15]:
simp50b = simp50[['MOL_ID', 'WDPAID', 'WDPA_PID', 'NAME', 'geometry']]
wdpa2 = wdpa_updated[['MOL_ID','AREA_KM2', 'DESIG', 'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'ORIG_NA', 'STATUS', 'STATUS_', 'GOV_TYP','nspecies']]

In [16]:
simp50b = simp50b.merge(wdpa2, how='left', on = 'MOL_ID')
simp50b.columns

Index(['MOL_ID', 'WDPAID', 'WDPA_PID', 'NAME', 'geometry', 'AREA_KM2', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'ORIG_NA', 'STATUS', 'STATUS_',
       'GOV_TYP', 'nspecies'],
      dtype='object')

In [18]:
# simp50b.to_file('/Users/sofia/Documents/HE_Data/WDPA/Simplifications/WDPA_Ter_Simp50_nspecies.geojson', driver='GeoJSON')
simp50b.to_file('/Users/sofia/Documents/HE_Data/WDPA/Simplifications/WDPA_Ter_Simp50_nspecies.shp', driver='ESRI Shapefile')

In [24]:
simp90 = gpd.read_file('/Users/sofia/Documents/HE_Data/WDPA/Simplifications/WDPA_Ter_Simp90/WDPA_Ter_Simp90.shp')
simp90b = simp90[['MOL_ID', 'WDPAID', 'WDPA_PID', 'NAME', 'geometry']]
simp90b = simp90b.merge(wdpa2, how='left', on = 'MOL_ID')
simp90b.columns

Index(['MOL_ID', 'WDPAID', 'WDPA_PID', 'NAME', 'geometry', 'AREA_KM2', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'ORIG_NA', 'STATUS', 'STATUS_',
       'GOV_TYP', 'nspecies'],
      dtype='object')

In [25]:
simp90b.to_file('/Users/sofia/Documents/HE_Data/WDPA/Simplifications/WDPA_Ter_Simp90_nspecies.geojson', driver='GeoJSON')

In [26]:
simp99 = gpd.read_file('/Users/sofia/Documents/HE_Data/WDPA/Simplifications/WDPA_Ter_Simp99/WDPA_Ter_Simp99.shp')
simp99b = simp99[['MOL_ID', 'WDPAID', 'WDPA_PID', 'NAME', 'geometry']]
simp99b = simp99b.merge(wdpa2, how='left', on = 'MOL_ID')
simp99b.columns

Index(['MOL_ID', 'WDPAID', 'WDPA_PID', 'NAME', 'geometry', 'AREA_KM2', 'DESIG',
       'DESIG_E', 'DESIG_T', 'IUCN_CA', 'ISO3', 'ORIG_NA', 'STATUS', 'STATUS_',
       'GOV_TYP', 'nspecies'],
      dtype='object')

In [27]:
simp99b.to_file('/Users/sofia/Documents/HE_Data/WDPA/Simplifications/WDPA_Ter_Simp99_nspecies.geojson', driver='GeoJSON')

## Future places

In [4]:
places = gpd.read_file('/Users/sofia/Documents/HE_Data/PlacesHE/Outputs/top_places_precalculated_20220601.geojson')

In [5]:
places.head(1)

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,group,...,mammals,reptiles,majority_land_cover_climate_regime,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,percentage_protected,geometry
0,1,AFG,Afghanistan,1,60.222222,7687.50492,1,442334.303716,1.124109e+10,1,...,"[ { ""SliceNumber"": 167.0, ""per_global"": 0.01, ...","[ { ""SliceNumber"": 9.0, ""per_global"": 0.289999...",184.0,26.31,57.02,15.5,0.0,2.092001e+06,0.0,"POLYGON ((7827944.398 4014125.022, 7814867.987..."


In [6]:
places.columns

Index(['OBJECTID', 'GID_0', 'NAME_0', 'cluster', 'PRmean', 'AREA_KM2',
       'MOL_ID', 'Shape_Leng', 'Shape_Area', 'group', 'FID', 'amphibians',
       'birds', 'mammals', 'reptiles', 'majority_land_cover_climate_regime',
       'percent_irrigated', 'percent_rainfed', 'percent_rangeland',
       'percent_urban', 'population_sum', 'percentage_protected', 'geometry'],
      dtype='object')

In [7]:
# Get data for all taxa
amphibians = pd.read_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Sample_amph_nozeroes.csv')
birds = pd.read_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Sample_birds_nozeroes.csv')
mammals = pd.read_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Sample_mammals_nozeroes.csv')
reptiles = pd.read_csv('/Users/sofia/Documents/HE_Data/PlacesHE/Inputs/Sample_reptiles_nozeroes.csv')

In [8]:
amphibians.head()

,OID_,MOL_ID,X,Y,amphibians,SliceNumber,Dimensions
0,819,124,1.044409e+06,5.381166e+06,34.0,819.0,SliceNumber
1,956,124,1.044409e+06,5.381166e+06,1257.0,956.0,SliceNumber
2,963,124,1.044409e+06,5.381166e+06,2758.0,963.0,SliceNumber
3,2340,124,1.044409e+06,5.381166e+06,1515.0,2340.0,SliceNumber
4,2764,124,1.044409e+06,5.381166e+06,2883.0,2764.0,SliceNumber


In [9]:
amph = amphibians.groupby('MOL_ID')['SliceNumber'].count().astype(int)
bir = birds.groupby('MOL_ID')['SliceNumber'].count().astype(int)
mamm = mammals.groupby('MOL_ID')['SliceNumber'].count().astype(int)
rept = reptiles.groupby('MOL_ID')['SliceNumber'].count().astype(int)

In [10]:
frame = { 'Amphibians': amph, 'Birds': bir, 'Mammals': mamm, 'Reptiles': rept }
df = pd.DataFrame(frame).reset_index()
cols = ['Amphibians', 'Birds', 'Mammals', 'Reptiles']
df[cols] = df[cols].fillna(0)
df[cols] = df[cols].astype('int')
df['nspecies'] = df['Amphibians'] + df['Birds'] + df['Mammals'] + df['Reptiles']
df

,MOL_ID,Amphibians,Birds,Mammals,Reptiles,nspecies
0,1,5,174,65,52,296
1,2,4,192,66,32,294
2,3,2,144,48,33,227
3,4,0,67,27,18,112
4,5,0,62,23,15,100
...,...,...,...,...,...,...
1911,1921,34,364,100,82,580
1912,1922,26,354,89,84,553
1913,1923,34,414,112,89,649
1914,1924,26,326,93,76,521


In [11]:
df[df.nspecies==0]

,MOL_ID,Amphibians,Birds,Mammals,Reptiles,nspecies


In [15]:
places_updated = places.merge(df[['nspecies', 'MOL_ID']], how='left', on = 'MOL_ID')
places_updated

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,group,...,reptiles,majority_land_cover_climate_regime,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,percentage_protected,geometry,nspecies
0,1,AFG,Afghanistan,1,60.222222,7687.504920,1,442334.303716,1.124109e+10,1,...,"[ { ""SliceNumber"": 9.0, ""per_global"": 0.289999...",184.0,26.31,57.02,15.50,0.00,2.092001e+06,0.00,"POLYGON ((7827944.398 4014125.022, 7814867.987...",296.0
1,2,AFG,Afghanistan,2,60.222222,4706.088317,2,354254.657577,7.043484e+09,1,...,"[ { ""SliceNumber"": 9.0, ""per_global"": 0.200000...",176.0,1.56,55.09,5.24,0.00,6.288113e+05,0.13,"POLYGON ((7884416.986 4117164.050, 7880408.999...",294.0
2,3,AFG,Afghanistan,3,52.312500,11768.366965,3,647899.188567,1.746601e+10,1,...,"[ { ""SliceNumber"": 1.0, ""per_global"": 0.63, ""p...",142.0,33.07,6.93,111.57,2.66,6.122301e+06,0.03,"POLYGON ((7661980.648 3995094.171, 7659753.669...",227.0
3,4,AFG,Afghanistan,4,45.500000,205.123344,4,63354.536671,2.958675e+08,1,...,"[ { ""SliceNumber"": 1.0, ""per_global"": 0.100000...",142.0,0.88,0.00,1.77,0.00,3.773772e+03,0.71,"POLYGON ((7551032.457 3954283.587, 7548339.754...",112.0
4,5,AFG,Afghanistan,5,45.500000,205.123344,5,63647.846097,2.979331e+08,1,...,"[ { ""SliceNumber"": 1.0, ""per_global"": 0.080000...",142.0,0.00,0.00,2.64,0.00,1.882826e+03,0.00,"POLYGON ((7523202.584 3994616.263, 7520509.882...",100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,1796,URY,Uruguay,1800,62.000000,205.123344,1800,61984.283090,2.861840e+08,9,...,"[ { ""SliceNumber"": 49.0, ""per_global"": 0.02, ""...",187.0,49.10,10.71,14.99,0.00,9.704867e+05,0.00,"POLYGON ((-6169094.783 -3775236.665, -6171787....",381.0
1917,1797,UZB,Uzbekistan,1801,82.000000,205.123344,1801,71300.429216,3.510649e+08,9,...,"[ { ""SliceNumber"": 322.0, ""per_global"": 0.02, ...",187.0,59.49,1.77,9.31,2.68,2.342898e+06,0.00,"POLYGON ((6938775.258 4870286.877, 6936082.555...",165.0
1918,1798,UZB,Uzbekistan,1802,47.222222,6250.745122,1802,476428.050484,1.072342e+10,9,...,"[ { ""SliceNumber"": 5.0, ""per_global"": 2.02, ""p...",187.0,39.38,2.94,12.61,1.31,3.457537e+06,0.69,"POLYGON ((7969630.322 4820536.531, 7965575.423...",235.0
1919,1799,UZB,Uzbekistan,1803,39.500000,862.495051,1803,132210.652673,1.385523e+09,9,...,"[ { ""SliceNumber"": 5.0, ""per_global"": 0.419999...",187.0,2.86,0.00,0.05,0.00,1.298762e+05,0.00,"POLYGON ((7597555.247 4577904.570, 7605992.591...",204.0


In [16]:
places_updated[places_updated.nspecies.isna()]

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,group,...,reptiles,majority_land_cover_climate_regime,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,percentage_protected,geometry,nspecies
62,63,ATA,Antarctica,64,10.380952,15523.349652,64,2.420112e+06,1.196205e+11,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((17209301.933 -10987109.861, 17206365...",NaN
69,70,ATA,Antarctica,72,4.107143,42832.512726,72,2.635374e+06,4.572966e+11,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-10631862.003 -11827308.546, -106320...",NaN
70,71,ATA,Antarctica,74,3.566667,22642.029254,74,2.571832e+06,3.795913e+11,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-16131294.799 -13691825.346, -161336...",NaN
71,72,ATA,Antarctica,75,3.111111,17204.480956,75,2.556923e+06,2.790347e+11,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-7450093.370 -13672296.708, -7451391...",NaN
74,75,ATA,Antarctica,78,2.862069,14746.897483,78,4.650649e+06,3.053829e+11,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"POLYGON ((-12457335.609 -14938844.810, -124588...",NaN


In [17]:
places_updated['nspecies'] = places_updated['nspecies'].fillna(0)
places_updated['nspecies'] = places_updated['nspecies'].astype('int')

In [20]:
places_updated[places_updated.cluster==1540]

,OBJECTID,GID_0,NAME_0,cluster,PRmean,AREA_KM2,MOL_ID,Shape_Leng,Shape_Area,group,...,reptiles,majority_land_cover_climate_regime,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum,percentage_protected,geometry,nspecies
1656,1536,ESP,Spain,1540,48.833333,205.123344,1540,76969.163053,3.896560e+08,8,...,"[ { ""SliceNumber"": 63.0, ""per_global"": 0.02, ""...",176.0,0.0,0.49,0.0,0.1,164208.803601,0.2,"POLYGON ((-881418.971 5371794.309, -884111.673...",186


In [21]:
places_updated.to_file('/Users/sofia/Documents/HE_Data/PlacesHE/Outputs/top_places_precalculated_nspecies.geojson', driver='GeoJSON')